## Import some dependencies

In [1]:
# Set the seed value for the notebook so the results are reproducible
import numpy as np
from numpy.random import seed
seed(1)
#wanted to take a look
np.random.rand(3)

array([4.17022005e-01, 7.20324493e-01, 1.14374817e-04])

In [2]:
import pandas as pd
import os

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

Setting up multiworker runs would be a bit more work here, see: https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras

## Read in and clean up data

In [4]:
#Import test data file, add columns for log-transformed data
test_data = pd.read_csv(os.path.join('../data', 'cleanLoanDataValidationAllIncome.csv'))
test_data['LogTotalIncome']=np.log(test_data['TotalIncome'])
test_data["LogLoanAmount"]= np.log(test_data["LoanAmount"])
pd.set_option('display.max_columns', None)
test_data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,TotalIncome,LogTotalIncome,LogLoanAmount
0,0,1,0,1,0,5720.0,0.0,110000.0,360.0,1,2,5720.0,8.651724,11.608236
1,0,1,1,1,0,3076.0,1500.0,126000.0,360.0,1,2,4576.0,8.428581,11.744037
2,0,1,2,1,0,5000.0,1800.0,208000.0,360.0,1,2,6800.0,8.824678,12.245293
3,0,1,2,1,0,2340.0,2546.0,100000.0,360.0,0,2,4886.0,8.494129,11.512925
4,0,0,0,0,0,3276.0,0.0,78000.0,360.0,1,2,3276.0,8.094378,11.264464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,0,1,3,0,1,4009.0,1777.0,113000.0,360.0,1,2,5786.0,8.663196,11.635143
341,0,1,0,1,0,4158.0,709.0,115000.0,360.0,1,2,4867.0,8.490233,11.652687
342,0,0,0,1,0,3250.0,1993.0,126000.0,360.0,0,1,5243.0,8.564649,11.744037
343,0,1,0,1,0,5000.0,2393.0,158000.0,360.0,1,0,7393.0,8.908289,11.970350


In [5]:
#Import training data file, add columns for log-transformed data
train_data = pd.read_csv(os.path.join('../data', 'cleanLoanDataTrainAllIncome.csv'))
train_data['LogTotalIncome']=np.log(train_data['TotalIncome'])
train_data["LogLoanAmount"]= np.log(train_data["LoanAmount"])
pd.set_option('display.max_columns', None)
train_data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome,LogTotalIncome,LogLoanAmount
0,0,1,1,1,0,4583.0,1508.0,128000.0,360.0,1,0,0,6091.0,8.714568,11.759786
1,0,1,0,1,1,3000.0,0.0,66000.0,360.0,1,2,1,3000.0,8.006368,11.097410
2,0,1,0,0,0,2583.0,2358.0,120000.0,360.0,1,2,1,4941.0,8.505323,11.695247
3,0,0,0,1,0,6000.0,0.0,141000.0,360.0,1,2,1,6000.0,8.699515,11.856515
4,0,1,2,1,1,5417.0,4196.0,267000.0,360.0,1,2,1,9613.0,9.170872,12.495004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,1,0,0,1,0,2900.0,0.0,71000.0,360.0,1,0,1,2900.0,7.972466,11.170435
559,0,1,3,1,0,4106.0,0.0,40000.0,180.0,1,0,1,4106.0,8.320205,10.596635
560,0,1,1,1,0,8072.0,240.0,253000.0,360.0,1,2,1,8312.0,9.025456,12.441145
561,0,1,2,1,0,7583.0,0.0,187000.0,360.0,1,2,1,7583.0,8.933664,12.138864


## 1a: Total income

In [6]:
X = train_data.drop(["Loan_Status", "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'LogTotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 10) (563,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [11]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=10))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [12]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.5776 - accuracy: 0.7014
Epoch 2/60
422/422 - 0s - loss: 0.5474 - accuracy: 0.7227
Epoch 3/60
422/422 - 0s - loss: 0.5396 - accuracy: 0.7678
Epoch 4/60
422/422 - 0s - loss: 0.5292 - accuracy: 0.7678
Epoch 5/60
422/422 - 0s - loss: 0.5234 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5179 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5182 - accuracy: 0.7701
Epoch 8/60
422/422 - 0s - loss: 0.5115 - accuracy: 0.7773
Epoch 9/60
422/422 - 0s - loss: 0.5121 - accuracy: 0.7749
Epoch 10/60
422/422 - 0s - loss: 0.5118 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5101 - accuracy: 0.7773
Epoch 12/60
422/422 - 0s - loss: 0.5033 - accuracy: 0.7796
Epoch 13/60
422/422 - 0s - loss: 0.5020 - accuracy: 0.7749
Epoch 14/60
422/422 - 0s - loss: 0.4994 - accuracy: 0.7773
Epoch 15/60
422/422 - 0s - loss: 0.4971 - accuracy: 0.7796
Epoch 16/60
422/422 - 0s - loss: 0.4965 - accuracy: 0.7820
Epoch 17/60
422/422 - 0s - loss: 0.4953 - ac

In [15]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 1a - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5360 - accuracy: 0.7021
Normal Neural Network 1a - Loss: 0.6198194506743275, Accuracy: 0.7021276354789734


1a results

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
=================================================================
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
    
141/1 - 0s - loss: 0.5977 - accuracy: 0.7021
Normal Neural Network 1a - Loss: 0.6939695828349878, Accuracy: 0.7021276354789734

## 1b: Separate income

In [16]:
X = train_data.drop(["Loan_Status", "TotalIncome", "LogLoanAmount", 'LogTotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 11) (563,)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [18]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [20]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [21]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=11))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [22]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               1200      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 11,502
Trainable params: 11,502
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6172 - accuracy: 0.6777
Epoch 2/60
422/422 - 0s - loss: 0.5673 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5486 - accuracy: 0.7085
Epoch 4/60
422/422 - 0s - loss: 0.5345 - accuracy: 0.7488
Epoch 5/60
422/422 - 0s - loss: 0.5227 - accuracy: 0.7536
Epoch 6/60
422/422 - 0s - loss: 0.5197 - accuracy: 0.7654
Epoch 7/60
422/422 - 0s - loss: 0.5107 - accuracy: 0.7701
Epoch 8/60
422/422 - 0s - loss: 0.5086 - accuracy: 0.7725
Epoch 9/60
422/422 - 0s - loss: 0.5051 - accuracy: 0.7701
Epoch 10/60
422/422 - 0s - loss: 0.5042 - accuracy: 0.7701
Epoch 11/60
422/422 - 0s - loss: 0.4995 - accuracy: 0.7725
Epoch 12/60
422/422 - 0s - loss: 0.4958 - accuracy: 0.7820
Epoch 13/60
422/422 - 0s - loss: 0.4969 - accuracy: 0.7844
Epoch 14/60
422/422 - 0s - loss: 0.4977 - accuracy: 0.7796
Epoch 15/60
422/422 - 0s - loss: 0.4979 - accuracy: 0.7796
Epoch 16/60
422/422 - 0s - loss: 0.4996 - accuracy: 0.7867
Epoch 17/60
422/422 - 0s - loss: 0.4836 - ac

In [25]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 1b - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.6425 - accuracy: 0.6809
Normal Neural Network 1b - Loss: 0.7497691723471838, Accuracy: 0.6808510422706604


1b results
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_16 (Dense)             (None, 100)               1200      
_________________________________________________________________
dense_17 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,502
Trainable params: 11,502
Non-trainable params: 0

141/1 - 0s - loss: 0.5967 - accuracy: 0.6879
Normal Neural Network 1 - Loss: 0.6984488072124779, Accuracy: 0.6879432797431946

## 1c: log(TotalIncome)

In [26]:
X = train_data.drop(["Loan_Status", "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'TotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 10) (563,)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [28]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [30]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [31]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=10))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [32]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               1100      
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 202       
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.5807 - accuracy: 0.7014
Epoch 2/60
422/422 - 0s - loss: 0.5512 - accuracy: 0.7109
Epoch 3/60
422/422 - 0s - loss: 0.5360 - accuracy: 0.7417
Epoch 4/60
422/422 - 0s - loss: 0.5287 - accuracy: 0.7630
Epoch 5/60
422/422 - 0s - loss: 0.5207 - accuracy: 0.7796
Epoch 6/60
422/422 - 0s - loss: 0.5135 - accuracy: 0.7725
Epoch 7/60
422/422 - 0s - loss: 0.5129 - accuracy: 0.7725
Epoch 8/60
422/422 - 0s - loss: 0.5090 - accuracy: 0.7820
Epoch 9/60
422/422 - 0s - loss: 0.5117 - accuracy: 0.7749
Epoch 10/60
422/422 - 0s - loss: 0.5061 - accuracy: 0.7773
Epoch 11/60
422/422 - 0s - loss: 0.5010 - accuracy: 0.7773
Epoch 12/60
422/422 - 0s - loss: 0.5004 - accuracy: 0.7796
Epoch 13/60
422/422 - 0s - loss: 0.4959 - accuracy: 0.7796
Epoch 14/60
422/422 - 0s - loss: 0.4941 - accuracy: 0.7820
Epoch 15/60
422/422 - 0s - loss: 0.4926 - accuracy: 0.7773
Epoch 16/60
422/422 - 0s - loss: 0.4893 - accuracy: 0.7796
Epoch 17/60
422/422 - 0s - loss: 0.4888 - ac

In [35]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 1c - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.6126 - accuracy: 0.6809
Normal Neural Network 1c - Loss: 0.7134639405189677, Accuracy: 0.6808510422706604


Run 1c result

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_6 (Dense)              (None, 100)               1100      
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 202       
=================================================================
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
    
141/1 - 0s - loss: 0.5485 - accuracy: 0.6950
Normal Neural Network 1c - Loss: 0.6729868354526818, Accuracy: 0.695035457611084


## 1d: log(TotalIncome), log(LoanAmount)

In [36]:
X = train_data.drop(["Loan_Status", "ApplicantIncome", "CoapplicantIncome", "LoanAmount", 'TotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 10) (563,)


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [38]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [40]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [41]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=10))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [42]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               1100      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 202       
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6341 - accuracy: 0.6540
Epoch 2/60
422/422 - 0s - loss: 0.5800 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5578 - accuracy: 0.7227
Epoch 4/60
422/422 - 0s - loss: 0.5426 - accuracy: 0.7393
Epoch 5/60
422/422 - 0s - loss: 0.5276 - accuracy: 0.7630
Epoch 6/60
422/422 - 0s - loss: 0.5197 - accuracy: 0.7749
Epoch 7/60
422/422 - 0s - loss: 0.5146 - accuracy: 0.7725
Epoch 8/60
422/422 - 0s - loss: 0.5119 - accuracy: 0.7773
Epoch 9/60
422/422 - 0s - loss: 0.5131 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5096 - accuracy: 0.7820
Epoch 11/60
422/422 - 0s - loss: 0.5098 - accuracy: 0.7773
Epoch 12/60
422/422 - 0s - loss: 0.5073 - accuracy: 0.7725
Epoch 13/60
422/422 - 0s - loss: 0.5070 - accuracy: 0.7844
Epoch 14/60
422/422 - 0s - loss: 0.5006 - accuracy: 0.7820
Epoch 15/60
422/422 - 0s - loss: 0.4975 - accuracy: 0.7796
Epoch 16/60
422/422 - 0s - loss: 0.4987 - accuracy: 0.7773
Epoch 17/60
422/422 - 0s - loss: 0.4975 - ac

In [45]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 1d - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5472 - accuracy: 0.7163
Normal Neural Network 1d - Loss: 0.6274816939171325, Accuracy: 0.716312050819397


Run 1d results

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_9 (Dense)              (None, 100)               1100      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
    
141/1 - 0s - loss: 0.5523 - accuracy: 0.7092
Normal Neural Network 1d - Loss: 0.6431249925430785, Accuracy: 0.7092198729515076

## 2a: take out Loan_Amount_Term column

In [46]:
X = train_data.drop(["Loan_Status", 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'LogTotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 9) (563,)


In [47]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [48]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [49]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [50]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [51]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=9))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [52]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 202       
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6243 - accuracy: 0.6493
Epoch 2/60
422/422 - 0s - loss: 0.5652 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5452 - accuracy: 0.7227
Epoch 4/60
422/422 - 0s - loss: 0.5325 - accuracy: 0.7701
Epoch 5/60
422/422 - 0s - loss: 0.5263 - accuracy: 0.7867
Epoch 6/60
422/422 - 0s - loss: 0.5281 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5194 - accuracy: 0.7749
Epoch 8/60
422/422 - 0s - loss: 0.5142 - accuracy: 0.7749
Epoch 9/60
422/422 - 0s - loss: 0.5114 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5139 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5097 - accuracy: 0.7796
Epoch 12/60
422/422 - 0s - loss: 0.5053 - accuracy: 0.7773
Epoch 13/60
422/422 - 0s - loss: 0.5050 - accuracy: 0.7701
Epoch 14/60
422/422 - 0s - loss: 0.5010 - accuracy: 0.7773
Epoch 15/60
422/422 - 0s - loss: 0.5012 - accuracy: 0.7773
Epoch 16/60
422/422 - 0s - loss: 0.4982 - accuracy: 0.7773
Epoch 17/60
422/422 - 0s - loss: 0.4981 - ac

In [55]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 2a - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5261 - accuracy: 0.6950
Normal Neural Network 2a - Loss: 0.6599454976988177, Accuracy: 0.695035457611084


Run 2a results
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_12 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0

141/1 - 0s - loss: 0.5610 - accuracy: 0.6950
Normal Neural Network 2a - Loss: 0.6652026197589036, Accuracy: 0.695035457611084


## 2b: Separate income, take out Loan_Amount_Term column

In [56]:
X = train_data.drop(["Loan_Status", 'Loan_Amount_Term', "TotalIncome", "LogLoanAmount", 'LogTotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 10) (563,)


In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [58]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [59]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [60]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [61]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=10))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [62]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [63]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 202       
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6123 - accuracy: 0.6967
Epoch 2/60
422/422 - 0s - loss: 0.5638 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5405 - accuracy: 0.7180
Epoch 4/60
422/422 - 0s - loss: 0.5278 - accuracy: 0.7725
Epoch 5/60
422/422 - 0s - loss: 0.5221 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5165 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5166 - accuracy: 0.7678
Epoch 8/60
422/422 - 0s - loss: 0.5139 - accuracy: 0.7773
Epoch 9/60
422/422 - 0s - loss: 0.5097 - accuracy: 0.7773
Epoch 10/60
422/422 - 0s - loss: 0.5106 - accuracy: 0.7725
Epoch 11/60
422/422 - 0s - loss: 0.5101 - accuracy: 0.7796
Epoch 12/60
422/422 - 0s - loss: 0.5005 - accuracy: 0.7725
Epoch 13/60
422/422 - 0s - loss: 0.5016 - accuracy: 0.7749
Epoch 14/60
422/422 - 0s - loss: 0.4983 - accuracy: 0.7773
Epoch 15/60
422/422 - 0s - loss: 0.4963 - accuracy: 0.7773
Epoch 16/60
422/422 - 0s - loss: 0.5029 - accuracy: 0.7820
Epoch 17/60
422/422 - 0s - loss: 0.5062 - ac

In [65]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 2b - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.6408 - accuracy: 0.6879
Normal Neural Network 2b - Loss: 0.7477878869002592, Accuracy: 0.6879432797431946


Run 2b Results

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_15 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
    
141/1 - 0s - loss: 0.7053 - accuracy: 0.6738
Normal Neural Network 2b - Loss: 0.7237191187574509, Accuracy: 0.673758864402771

## 2c: take out Loan_Amount_Term column, use log(TotalIncome)

In [66]:
X = train_data.drop(["Loan_Status", 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'TotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 9) (563,)


In [67]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [68]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [69]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [70]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [71]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=9))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [72]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [73]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_19 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 202       
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6070 - accuracy: 0.6991
Epoch 2/60
422/422 - 0s - loss: 0.5751 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5491 - accuracy: 0.7038
Epoch 4/60
422/422 - 0s - loss: 0.5360 - accuracy: 0.7417
Epoch 5/60
422/422 - 0s - loss: 0.5242 - accuracy: 0.7654
Epoch 6/60
422/422 - 0s - loss: 0.5198 - accuracy: 0.7749
Epoch 7/60
422/422 - 0s - loss: 0.5161 - accuracy: 0.7701
Epoch 8/60
422/422 - 0s - loss: 0.5125 - accuracy: 0.7773
Epoch 9/60
422/422 - 0s - loss: 0.5121 - accuracy: 0.7773
Epoch 10/60
422/422 - 0s - loss: 0.5056 - accuracy: 0.7773
Epoch 11/60
422/422 - 0s - loss: 0.5035 - accuracy: 0.7749
Epoch 12/60
422/422 - 0s - loss: 0.5040 - accuracy: 0.7749
Epoch 13/60
422/422 - 0s - loss: 0.4997 - accuracy: 0.7773
Epoch 14/60
422/422 - 0s - loss: 0.4989 - accuracy: 0.7773
Epoch 15/60
422/422 - 0s - loss: 0.5029 - accuracy: 0.7725
Epoch 16/60
422/422 - 0s - loss: 0.5110 - accuracy: 0.7749
Epoch 17/60
422/422 - 0s - loss: 0.4957 - ac

In [75]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 2c - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5430 - accuracy: 0.6950
Normal Neural Network 2c - Loss: 0.660601813319727, Accuracy: 0.695035457611084


Run 2c results

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_18 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_19 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
    
141/1 - 0s - loss: 0.6211 - accuracy: 0.7163
Normal Neural Network 2c - Loss: 0.6821013312813238, Accuracy: 0.716312050819397

## 2d: take out Loan_Amount_Term column, use log(TotalIncome), log(LoanAmount)

In [76]:
X = train_data.drop(["Loan_Status", 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LoanAmount", 'TotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 9) (563,)


In [77]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [78]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [79]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [80]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [81]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=9))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [82]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_22 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 202       
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
_________________________________________________________________


In [84]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6100 - accuracy: 0.6588
Epoch 2/60
422/422 - 0s - loss: 0.5685 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5500 - accuracy: 0.7133
Epoch 4/60
422/422 - 0s - loss: 0.5332 - accuracy: 0.7370
Epoch 5/60
422/422 - 0s - loss: 0.5233 - accuracy: 0.7773
Epoch 6/60
422/422 - 0s - loss: 0.5204 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5180 - accuracy: 0.7749
Epoch 8/60
422/422 - 0s - loss: 0.5097 - accuracy: 0.7773
Epoch 9/60
422/422 - 0s - loss: 0.5078 - accuracy: 0.7701
Epoch 10/60
422/422 - 0s - loss: 0.5020 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5072 - accuracy: 0.7725
Epoch 12/60
422/422 - 0s - loss: 0.5016 - accuracy: 0.7844
Epoch 13/60
422/422 - 0s - loss: 0.4976 - accuracy: 0.7773
Epoch 14/60
422/422 - 0s - loss: 0.4959 - accuracy: 0.7796
Epoch 15/60
422/422 - 0s - loss: 0.4939 - accuracy: 0.7773
Epoch 16/60
422/422 - 0s - loss: 0.4954 - accuracy: 0.7867
Epoch 17/60
422/422 - 0s - loss: 0.4883 - ac

In [85]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 2d - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5993 - accuracy: 0.6879
Normal Neural Network 2d - Loss: 0.6693894389673327, Accuracy: 0.6879432797431946


Run 2d results

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_21 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_22 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
    
141/1 - 0s - loss: 0.6568 - accuracy: 0.6809
Normal Neural Network 2d - Loss: 0.7042758709995459, Accuracy: 0.6808510422706604

## 3a: take out Loan_Amount_Term and LoanAmount columns

In [86]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'LogTotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 8) (563,)


In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [88]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [89]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [90]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [91]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=8))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [92]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [93]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_25 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 202       
Total params: 11,202
Trainable params: 11,202
Non-trainable params: 0
_________________________________________________________________


In [94]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6074 - accuracy: 0.6991
Epoch 2/60
422/422 - 0s - loss: 0.5659 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5459 - accuracy: 0.7393
Epoch 4/60
422/422 - 0s - loss: 0.5350 - accuracy: 0.7536
Epoch 5/60
422/422 - 0s - loss: 0.5286 - accuracy: 0.7607
Epoch 6/60
422/422 - 0s - loss: 0.5211 - accuracy: 0.7844
Epoch 7/60
422/422 - 0s - loss: 0.5200 - accuracy: 0.7749
Epoch 8/60
422/422 - 0s - loss: 0.5157 - accuracy: 0.7796
Epoch 9/60
422/422 - 0s - loss: 0.5130 - accuracy: 0.7725
Epoch 10/60
422/422 - 0s - loss: 0.5096 - accuracy: 0.7749
Epoch 11/60
422/422 - 0s - loss: 0.5078 - accuracy: 0.7749
Epoch 12/60
422/422 - 0s - loss: 0.5057 - accuracy: 0.7773
Epoch 13/60
422/422 - 0s - loss: 0.5047 - accuracy: 0.7773
Epoch 14/60
422/422 - 0s - loss: 0.5042 - accuracy: 0.7701
Epoch 15/60
422/422 - 0s - loss: 0.5009 - accuracy: 0.7820
Epoch 16/60
422/422 - 0s - loss: 0.5073 - accuracy: 0.7749
Epoch 17/60
422/422 - 0s - loss: 0.4969 - ac

In [95]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 3a - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5901 - accuracy: 0.7092
Normal Neural Network 3a - Loss: 0.6524173710363131, Accuracy: 0.7092198729515076


Run 3a results
_________________________________________________________________
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_24 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_25 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,202
Trainable params: 11,202
Non-trainable params: 0

141/1 - 0s - loss: 0.5328 - accuracy: 0.6879
Normal Neural Network 3a - Loss: 0.6187358534505182, Accuracy: 0.6879432797431946

## 3b: Separate income

In [96]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "TotalIncome", "LogLoanAmount", 'LogTotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 9) (563,)


In [97]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [98]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [99]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [100]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [101]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=9))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [102]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [103]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_28 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 202       
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
_________________________________________________________________


In [104]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.5857 - accuracy: 0.7062
Epoch 2/60
422/422 - 0s - loss: 0.5565 - accuracy: 0.7085
Epoch 3/60
422/422 - 0s - loss: 0.5396 - accuracy: 0.7417
Epoch 4/60
422/422 - 0s - loss: 0.5277 - accuracy: 0.7725
Epoch 5/60
422/422 - 0s - loss: 0.5209 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5165 - accuracy: 0.7773
Epoch 7/60
422/422 - 0s - loss: 0.5170 - accuracy: 0.7749
Epoch 8/60
422/422 - 0s - loss: 0.5133 - accuracy: 0.7773
Epoch 9/60
422/422 - 0s - loss: 0.5156 - accuracy: 0.7749
Epoch 10/60
422/422 - 0s - loss: 0.5101 - accuracy: 0.7773
Epoch 11/60
422/422 - 0s - loss: 0.5102 - accuracy: 0.7773
Epoch 12/60
422/422 - 0s - loss: 0.5049 - accuracy: 0.7773
Epoch 13/60
422/422 - 0s - loss: 0.5037 - accuracy: 0.7773
Epoch 14/60
422/422 - 0s - loss: 0.5015 - accuracy: 0.7773
Epoch 15/60
422/422 - 0s - loss: 0.4961 - accuracy: 0.7773
Epoch 16/60
422/422 - 0s - loss: 0.4955 - accuracy: 0.7773
Epoch 17/60
422/422 - 0s - loss: 0.4924 - ac

In [105]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 3b - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 1s - loss: 0.6967 - accuracy: 0.6596
Normal Neural Network 3b - Loss: 0.7645552128764754, Accuracy: 0.6595744490623474


Run 3b results

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_27 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_28 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
    
141/1 - 1s - loss: 0.6357 - accuracy: 0.6596
Normal Neural Network 3b - Loss: 0.6898306582836394, Accuracy: 0.6595744490623474

## 3c: take out Loan_Amount_Term and LoanAmount columns, use log(TotalIncome)

In [106]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'TotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 8) (563,)


In [107]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [108]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [109]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [110]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [111]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=8))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [112]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [113]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_31 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 202       
Total params: 11,202
Trainable params: 11,202
Non-trainable params: 0
_________________________________________________________________


In [114]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6173 - accuracy: 0.6848
Epoch 2/60
422/422 - 0s - loss: 0.5685 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5523 - accuracy: 0.7180
Epoch 4/60
422/422 - 0s - loss: 0.5432 - accuracy: 0.7370
Epoch 5/60
422/422 - 0s - loss: 0.5330 - accuracy: 0.7393
Epoch 6/60
422/422 - 0s - loss: 0.5245 - accuracy: 0.7749
Epoch 7/60
422/422 - 0s - loss: 0.5248 - accuracy: 0.7678
Epoch 8/60
422/422 - 0s - loss: 0.5238 - accuracy: 0.7749
Epoch 9/60
422/422 - 0s - loss: 0.5142 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5183 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5165 - accuracy: 0.7725
Epoch 12/60
422/422 - 0s - loss: 0.5074 - accuracy: 0.7678
Epoch 13/60
422/422 - 0s - loss: 0.5043 - accuracy: 0.7725
Epoch 14/60
422/422 - 0s - loss: 0.5112 - accuracy: 0.7678
Epoch 15/60
422/422 - 0s - loss: 0.5143 - accuracy: 0.7796
Epoch 16/60
422/422 - 0s - loss: 0.5007 - accuracy: 0.7820
Epoch 17/60
422/422 - 0s - loss: 0.4949 - ac

In [115]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 3c - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.6204 - accuracy: 0.6667
Normal Neural Network 3c - Loss: 0.6584342181259859, Accuracy: 0.6666666865348816


Run 3c results

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_30 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_31 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 202       
=================================================================
Total params: 11,202
Trainable params: 11,202
Non-trainable params: 0
    
141/1 - 0s - loss: 0.5261 - accuracy: 0.7234
Normal Neural Network 3c - Loss: 0.651864561628788, Accuracy: 0.7234042286872864  

## 4a take out Loan_Amount_Term and LoanAmount columns, add a layer 

In [116]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'LogTotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 8) (563,)


In [117]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [118]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [119]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [120]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [121]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=8))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [122]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [123]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_34 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_35 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_36 (Dense)             (None, 2)                 202       
Total params: 21,302
Trainable params: 21,302
Non-trainable params: 0
_________________________________________________________________


In [124]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6166 - accuracy: 0.6588
Epoch 2/60
422/422 - 0s - loss: 0.5575 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5458 - accuracy: 0.7701
Epoch 4/60
422/422 - 0s - loss: 0.5283 - accuracy: 0.7678
Epoch 5/60
422/422 - 0s - loss: 0.5193 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5158 - accuracy: 0.7701
Epoch 7/60
422/422 - 0s - loss: 0.5116 - accuracy: 0.7749
Epoch 8/60
422/422 - 0s - loss: 0.5100 - accuracy: 0.7749
Epoch 9/60
422/422 - 0s - loss: 0.5037 - accuracy: 0.7749
Epoch 10/60
422/422 - 0s - loss: 0.5059 - accuracy: 0.7749
Epoch 11/60
422/422 - 0s - loss: 0.5015 - accuracy: 0.7915
Epoch 12/60
422/422 - 0s - loss: 0.4971 - accuracy: 0.7844
Epoch 13/60
422/422 - 0s - loss: 0.4941 - accuracy: 0.7867
Epoch 14/60
422/422 - 0s - loss: 0.4908 - accuracy: 0.7796
Epoch 15/60
422/422 - 0s - loss: 0.4867 - accuracy: 0.7867
Epoch 16/60
422/422 - 0s - loss: 0.4798 - accuracy: 0.7962
Epoch 17/60
422/422 - 0s - loss: 0.4825 - ac

In [125]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 4a - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 1.0925 - accuracy: 0.6738
Normal Neural Network 4a - Loss: 1.0454478035581873, Accuracy: 0.673758864402771


Run 4a results

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_33 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_34 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_35 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_36 (Dense)             (None, 2)                 202       
=================================================================
Total params: 21,302
Trainable params: 21,302
Non-trainable params: 0

141/1 - 0s - loss: 0.9865 - accuracy: 0.6667
Normal Neural Network 4a - Loss: 1.0510162102415206, Accuracy: 0.6666666865348816


## 4b: Separate income

In [126]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "TotalIncome", "LogLoanAmount", 'LogTotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 9) (563,)


In [127]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [128]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [129]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [130]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [131]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=9))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [132]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [133]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_38 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_39 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_40 (Dense)             (None, 2)                 202       
Total params: 21,402
Trainable params: 21,402
Non-trainable params: 0
_________________________________________________________________


In [134]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6164 - accuracy: 0.7014
Epoch 2/60
422/422 - 0s - loss: 0.5719 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5465 - accuracy: 0.7014
Epoch 4/60
422/422 - 0s - loss: 0.5357 - accuracy: 0.7512
Epoch 5/60
422/422 - 0s - loss: 0.5226 - accuracy: 0.7773
Epoch 6/60
422/422 - 0s - loss: 0.5175 - accuracy: 0.7749
Epoch 7/60
422/422 - 0s - loss: 0.5202 - accuracy: 0.7749
Epoch 8/60
422/422 - 0s - loss: 0.5183 - accuracy: 0.7749
Epoch 9/60
422/422 - 0s - loss: 0.5129 - accuracy: 0.7773
Epoch 10/60
422/422 - 0s - loss: 0.5138 - accuracy: 0.7749
Epoch 11/60
422/422 - 0s - loss: 0.5071 - accuracy: 0.7773
Epoch 12/60
422/422 - 0s - loss: 0.4994 - accuracy: 0.7773
Epoch 13/60
422/422 - 0s - loss: 0.4957 - accuracy: 0.7773
Epoch 14/60
422/422 - 0s - loss: 0.4894 - accuracy: 0.7773
Epoch 15/60
422/422 - 0s - loss: 0.4871 - accuracy: 0.7773
Epoch 16/60
422/422 - 0s - loss: 0.4838 - accuracy: 0.7820
Epoch 17/60
422/422 - 0s - loss: 0.4792 - ac

In [135]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 4b - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.8273 - accuracy: 0.6809
Normal Neural Network 4b - Loss: 0.8672996049231672, Accuracy: 0.6808510422706604


Run 4b results

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_37 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_38 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_39 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_40 (Dense)             (None, 2)                 202       
=================================================================
Total params: 21,402
Trainable params: 21,402
Non-trainable params: 0
    
141/1 - 0s - loss: 0.9863 - accuracy: 0.6667
Normal Neural Network 4b - Loss: 1.1121582591787298, Accuracy: 0.6666666865348816  

## 4c take out Loan_Amount_Term and LoanAmount columns, add a layer, used log(TotalIncome)

In [136]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'TotalIncome'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 8) (563,)


In [137]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [138]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [139]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [140]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [141]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=8))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [142]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [143]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_42 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_43 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 202       
Total params: 21,302
Trainable params: 21,302
Non-trainable params: 0
_________________________________________________________________


In [144]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 2s - loss: 0.6101 - accuracy: 0.7014
Epoch 2/60
422/422 - 0s - loss: 0.5644 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5467 - accuracy: 0.7559
Epoch 4/60
422/422 - 0s - loss: 0.5317 - accuracy: 0.7701
Epoch 5/60
422/422 - 0s - loss: 0.5172 - accuracy: 0.7725
Epoch 6/60
422/422 - 0s - loss: 0.5193 - accuracy: 0.7701
Epoch 7/60
422/422 - 0s - loss: 0.5175 - accuracy: 0.7749
Epoch 8/60
422/422 - 0s - loss: 0.5151 - accuracy: 0.7820
Epoch 9/60
422/422 - 0s - loss: 0.5029 - accuracy: 0.7867
Epoch 10/60
422/422 - 0s - loss: 0.5001 - accuracy: 0.7773
Epoch 11/60
422/422 - 0s - loss: 0.4963 - accuracy: 0.7867
Epoch 12/60
422/422 - 0s - loss: 0.4883 - accuracy: 0.7867
Epoch 13/60
422/422 - 0s - loss: 0.4855 - accuracy: 0.7820
Epoch 14/60
422/422 - 0s - loss: 0.4806 - accuracy: 0.7844
Epoch 15/60
422/422 - 0s - loss: 0.4760 - accuracy: 0.7938
Epoch 16/60
422/422 - 0s - loss: 0.4728 - accuracy: 0.7844
Epoch 17/60
422/422 - 0s - loss: 0.4716 - ac

In [145]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 4c - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 1.0077 - accuracy: 0.6667
Normal Neural Network 4c - Loss: 1.0052797371614064, Accuracy: 0.6666666865348816


Run 4c results

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_41 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_42 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_43 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 202       
=================================================================
Total params: 21,302
Trainable params: 21,302
Non-trainable params: 0
    
141/1 - 0s - loss: 1.2298 - accuracy: 0.6667
Normal Neural Network 4c - Loss: 1.0912111718603905, Accuracy: 0.6666666865348816

## 5a: 2 layers, Total Income, take out Loan_Amount_Term and LoanAmount columns, also remove columns with lower importance in Random Tree runs ( 'Gender', 'Education', 'Self_Employed', 'Married')

In [155]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'LogTotalIncome', 'Gender', 'Education', 'Self_Employed', 'Married'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 4) (563,)


In [156]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [157]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [158]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [159]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [160]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [161]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [162]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_49 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_50 (Dense)             (None, 2)                 202       
Total params: 10,802
Trainable params: 10,802
Non-trainable params: 0
_________________________________________________________________


In [163]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6180 - accuracy: 0.6848
Epoch 2/60
422/422 - 0s - loss: 0.5553 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5515 - accuracy: 0.7062
Epoch 4/60
422/422 - 0s - loss: 0.5399 - accuracy: 0.7370
Epoch 5/60
422/422 - 0s - loss: 0.5342 - accuracy: 0.7512
Epoch 6/60
422/422 - 0s - loss: 0.5312 - accuracy: 0.7630
Epoch 7/60
422/422 - 0s - loss: 0.5313 - accuracy: 0.7678
Epoch 8/60
422/422 - 0s - loss: 0.5312 - accuracy: 0.7678
Epoch 9/60
422/422 - 0s - loss: 0.5248 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5289 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5239 - accuracy: 0.7678
Epoch 12/60
422/422 - 0s - loss: 0.5266 - accuracy: 0.7678
Epoch 13/60
422/422 - 0s - loss: 0.5263 - accuracy: 0.7678
Epoch 14/60
422/422 - 0s - loss: 0.5231 - accuracy: 0.7678
Epoch 15/60
422/422 - 0s - loss: 0.5225 - accuracy: 0.7678
Epoch 16/60
422/422 - 0s - loss: 0.5223 - accuracy: 0.7678
Epoch 17/60
422/422 - 0s - loss: 0.5202 - ac

In [164]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 5a - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.4831 - accuracy: 0.7518
Normal Neural Network 5a - Loss: 0.5475983116643649, Accuracy: 0.7517730593681335


Run 5a results
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_48 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_49 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_50 (Dense)             (None, 2)                 202       
=================================================================
Total params: 10,802
Trainable params: 10,802
Non-trainable params: 0

141/1 - 0s - loss: 0.4831 - accuracy: 0.7518
Normal Neural Network 5a - Loss: 0.5475983116643649, Accuracy: 0.7517730593681335


## 5b: 2 layers, App, Co-App, take out Loan_Amount_Term and LoanAmount columns, also remove columns with lower importance in Random Tree runs ( 'Gender', 'Education', 'Self_Employed', 'Married')

In [165]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "TotalIncome", "LogLoanAmount", 'LogTotalIncome', 'Gender', 'Education', 'Self_Employed', 'Married'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 5) (563,)


In [166]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [167]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [168]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [169]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [170]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=5))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [171]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [172]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 100)               600       
_________________________________________________________________
dense_52 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_53 (Dense)             (None, 2)                 202       
Total params: 10,902
Trainable params: 10,902
Non-trainable params: 0
_________________________________________________________________


In [173]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6010 - accuracy: 0.6967
Epoch 2/60
422/422 - 0s - loss: 0.5539 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5391 - accuracy: 0.7393
Epoch 4/60
422/422 - 0s - loss: 0.5343 - accuracy: 0.7678
Epoch 5/60
422/422 - 0s - loss: 0.5279 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5258 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5249 - accuracy: 0.7678
Epoch 8/60
422/422 - 0s - loss: 0.5224 - accuracy: 0.7678
Epoch 9/60
422/422 - 0s - loss: 0.5267 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5206 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5212 - accuracy: 0.7678
Epoch 12/60
422/422 - 0s - loss: 0.5192 - accuracy: 0.7678
Epoch 13/60
422/422 - 0s - loss: 0.5183 - accuracy: 0.7678
Epoch 14/60
422/422 - 0s - loss: 0.5189 - accuracy: 0.7678
Epoch 15/60
422/422 - 0s - loss: 0.5178 - accuracy: 0.7678
Epoch 16/60
422/422 - 0s - loss: 0.5176 - accuracy: 0.7678
Epoch 17/60
422/422 - 0s - loss: 0.5180 - ac

In [174]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 5b - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5026 - accuracy: 0.7730
Normal Neural Network 5b - Loss: 0.5635992305498596, Accuracy: 0.7730496525764465


Run 5b results

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_51 (Dense)             (None, 100)               600       
_________________________________________________________________
dense_52 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_53 (Dense)             (None, 2)                 202       
=================================================================
Total params: 10,902
Trainable params: 10,902
Non-trainable params: 0

141/1 - 0s - loss: 0.5026 - accuracy: 0.7730
Normal Neural Network 5b - Loss: 0.5635992305498596, Accuracy: 0.7730496525764465

## 5c: 2 layers, log(Total Income), take out Loan_Amount_Term and LoanAmount columns, also remove columns with lower importance in Random Tree runs ( 'Gender', 'Education', 'Self_Employed', 'Married')

In [175]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'TotalIncome', 'Gender', 'Education', 'Self_Employed', 'Married'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 4) (563,)


In [176]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [177]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [178]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [179]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [180]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [181]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [182]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_55 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_56 (Dense)             (None, 2)                 202       
Total params: 10,802
Trainable params: 10,802
Non-trainable params: 0
_________________________________________________________________


In [183]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6545 - accuracy: 0.6374
Epoch 2/60
422/422 - 0s - loss: 0.5757 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5499 - accuracy: 0.7133
Epoch 4/60
422/422 - 0s - loss: 0.5402 - accuracy: 0.7536
Epoch 5/60
422/422 - 0s - loss: 0.5324 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5304 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5283 - accuracy: 0.7678
Epoch 8/60
422/422 - 0s - loss: 0.5275 - accuracy: 0.7678
Epoch 9/60
422/422 - 0s - loss: 0.5276 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5241 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5249 - accuracy: 0.7678
Epoch 12/60
422/422 - 0s - loss: 0.5244 - accuracy: 0.7678
Epoch 13/60
422/422 - 0s - loss: 0.5237 - accuracy: 0.7678
Epoch 14/60
422/422 - 0s - loss: 0.5244 - accuracy: 0.7678
Epoch 15/60
422/422 - 0s - loss: 0.5237 - accuracy: 0.7678
Epoch 16/60
422/422 - 0s - loss: 0.5205 - accuracy: 0.7678
Epoch 17/60
422/422 - 0s - loss: 0.5233 - ac

In [184]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 5b - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.4681 - accuracy: 0.7589
Normal Neural Network 5b - Loss: 0.54210065358074, Accuracy: 0.758865237236023


Run 5c results

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_54 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_55 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_56 (Dense)             (None, 2)                 202       
=================================================================
Total params: 10,802
Trainable params: 10,802
Non-trainable params: 0

141/1 - 0s - loss: 0.4681 - accuracy: 0.7589
Normal Neural Network 5b - Loss: 0.54210065358074, Accuracy: 0.758865237236023

## 6a: 3 layers, Total Income, take out Loan_Amount_Term and LoanAmount columns, also remove columns with lower importance in Random Tree runs ( 'Gender', 'Education', 'Self_Employed', 'Married')

In [185]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'LogTotalIncome', 'Gender', 'Education', 'Self_Employed', 'Married'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 4) (563,)


In [186]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [187]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [188]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [189]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [190]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [191]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [192]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_58 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_59 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_60 (Dense)             (None, 2)                 202       
Total params: 20,902
Trainable params: 20,902
Non-trainable params: 0
_________________________________________________________________


In [193]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6049 - accuracy: 0.6896
Epoch 2/60
422/422 - 0s - loss: 0.5563 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5502 - accuracy: 0.7536
Epoch 4/60
422/422 - 0s - loss: 0.5340 - accuracy: 0.7678
Epoch 5/60
422/422 - 0s - loss: 0.5324 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5288 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5249 - accuracy: 0.7678
Epoch 8/60
422/422 - 0s - loss: 0.5228 - accuracy: 0.7678
Epoch 9/60
422/422 - 0s - loss: 0.5232 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5212 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5341 - accuracy: 0.7678
Epoch 12/60
422/422 - 0s - loss: 0.5244 - accuracy: 0.7678
Epoch 13/60
422/422 - 0s - loss: 0.5259 - accuracy: 0.7678
Epoch 14/60
422/422 - 0s - loss: 0.5223 - accuracy: 0.7678
Epoch 15/60
422/422 - 0s - loss: 0.5199 - accuracy: 0.7678
Epoch 16/60
422/422 - 0s - loss: 0.5207 - accuracy: 0.7678
Epoch 17/60
422/422 - 0s - loss: 0.5244 - ac

In [194]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 6a - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5371 - accuracy: 0.7305
Normal Neural Network 6a - Loss: 0.5986272792021433, Accuracy: 0.7304964661598206


6a results

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_57 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_58 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_59 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_60 (Dense)             (None, 2)                 202       
=================================================================
Total params: 20,902
Trainable params: 20,902
Non-trainable params: 0

141/1 - 0s - loss: 0.5371 - accuracy: 0.7305
Normal Neural Network 6a - Loss: 0.5986272792021433, Accuracy: 0.7304964661598206


## 6b: 3 layers, App, Co-App, take out Loan_Amount_Term and LoanAmount columns, also remove columns with lower importance in Random Tree runs ( 'Gender', 'Education', 'Self_Employed', 'Married')

In [195]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "TotalIncome", "LogLoanAmount", 'LogTotalIncome', 'Gender', 'Education', 'Self_Employed', 'Married'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 5) (563,)


In [196]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [197]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [198]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [199]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [200]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=5))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [201]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [202]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 100)               600       
_________________________________________________________________
dense_62 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_63 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_64 (Dense)             (None, 2)                 202       
Total params: 21,002
Trainable params: 21,002
Non-trainable params: 0
_________________________________________________________________


In [203]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.5944 - accuracy: 0.7014
Epoch 2/60
422/422 - 0s - loss: 0.5546 - accuracy: 0.7038
Epoch 3/60
422/422 - 0s - loss: 0.5574 - accuracy: 0.7654
Epoch 4/60
422/422 - 0s - loss: 0.5375 - accuracy: 0.7630
Epoch 5/60
422/422 - 0s - loss: 0.5326 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5287 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5280 - accuracy: 0.7678
Epoch 8/60
422/422 - 0s - loss: 0.5234 - accuracy: 0.7678
Epoch 9/60
422/422 - 0s - loss: 0.5187 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5217 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5193 - accuracy: 0.7678
Epoch 12/60
422/422 - 0s - loss: 0.5152 - accuracy: 0.7678
Epoch 13/60
422/422 - 0s - loss: 0.5181 - accuracy: 0.7678
Epoch 14/60
422/422 - 0s - loss: 0.5161 - accuracy: 0.7678
Epoch 15/60
422/422 - 0s - loss: 0.5163 - accuracy: 0.7701
Epoch 16/60
422/422 - 0s - loss: 0.5145 - accuracy: 0.7678
Epoch 17/60
422/422 - 0s - loss: 0.5157 - ac

In [204]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 6a - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5616 - accuracy: 0.7518
Normal Neural Network 6a - Loss: 0.6191868456542915, Accuracy: 0.7517730593681335


6b Results

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_61 (Dense)             (None, 100)               600       
_________________________________________________________________
dense_62 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_63 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_64 (Dense)             (None, 2)                 202       
=================================================================
Total params: 21,002
Trainable params: 21,002
Non-trainable params: 0

141/1 - 0s - loss: 0.5616 - accuracy: 0.7518
Normal Neural Network 6a - Loss: 0.6191868456542915, Accuracy: 0.7517730593681335

## 6c: 3 layers, log(Total Income), take out Loan_Amount_Term and LoanAmount columns, also remove columns with lower importance in Random Tree runs ( 'Gender', 'Education', 'Self_Employed', 'Married')

In [205]:
X = train_data.drop(["Loan_Status", 'LoanAmount', 'Loan_Amount_Term', "ApplicantIncome", "CoapplicantIncome", "LogLoanAmount", 'TotalIncome', 'Gender', 'Education', 'Self_Employed', 'Married'], axis=1)
y = train_data["Loan_Status"]
print(X.shape, y.shape)

(563, 4) (563,)


In [206]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [207]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [208]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [209]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [210]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [211]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [212]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_66 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_67 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_68 (Dense)             (None, 2)                 202       
Total params: 20,902
Trainable params: 20,902
Non-trainable params: 0
_________________________________________________________________


In [213]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Train on 422 samples
Epoch 1/60
422/422 - 1s - loss: 0.6491 - accuracy: 0.6043
Epoch 2/60
422/422 - 0s - loss: 0.5687 - accuracy: 0.7014
Epoch 3/60
422/422 - 0s - loss: 0.5420 - accuracy: 0.7417
Epoch 4/60
422/422 - 0s - loss: 0.5310 - accuracy: 0.7678
Epoch 5/60
422/422 - 0s - loss: 0.5330 - accuracy: 0.7678
Epoch 6/60
422/422 - 0s - loss: 0.5297 - accuracy: 0.7678
Epoch 7/60
422/422 - 0s - loss: 0.5253 - accuracy: 0.7678
Epoch 8/60
422/422 - 0s - loss: 0.5258 - accuracy: 0.7678
Epoch 9/60
422/422 - 0s - loss: 0.5224 - accuracy: 0.7678
Epoch 10/60
422/422 - 0s - loss: 0.5217 - accuracy: 0.7678
Epoch 11/60
422/422 - 0s - loss: 0.5247 - accuracy: 0.7701
Epoch 12/60
422/422 - 0s - loss: 0.5219 - accuracy: 0.7678
Epoch 13/60
422/422 - 0s - loss: 0.5196 - accuracy: 0.7678
Epoch 14/60
422/422 - 0s - loss: 0.5194 - accuracy: 0.7678
Epoch 15/60
422/422 - 0s - loss: 0.5191 - accuracy: 0.7678
Epoch 16/60
422/422 - 0s - loss: 0.5250 - accuracy: 0.7678
Epoch 17/60
422/422 - 0s - loss: 0.5186 - ac

In [214]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network 6a - Loss: {model_loss}, Accuracy: {model_accuracy}")

141/1 - 0s - loss: 0.5182 - accuracy: 0.7447
Normal Neural Network 6a - Loss: 0.5848448889475342, Accuracy: 0.7446808218955994


6c Results

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_65 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_66 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_67 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_68 (Dense)             (None, 2)                 202       
=================================================================
Total params: 20,902
Trainable params: 20,902
Non-trainable params: 0

141/1 - 0s - loss: 0.5182 - accuracy: 0.7447
Normal Neural Network 6a - Loss: 0.5848448889475342, Accuracy: 0.7446808218955994
